In [27]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder
    .appName('iot_data_ingestion')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/06/09 09:07:09 WARN Utils: Your hostname, HP-Pavilion-15-ec2121nr resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/09 09:07:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c089113-2781-41f3-8934-fd7549dcf913;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 216ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

In [28]:
sensors = [col for col in spark.read.format('delta').load('../data/bronze/bronze_incoming_data').columns if 'sensor' in col]
aggregations = [F.avg('energy').alias('avg_energy')]

for sensor in sensors:
    aggregations.append(F.stddev_pop(sensor).alias(f'std_{sensor}'))
    aggregations.append(F.percentile_approx(sensor, [0.1, 0.3, 0.6, 0.8, 0.95]).alias(f'percentiles_{sensor}'))

In [31]:
df = (
    spark.read.format('delta')
    .load('../data/bronze/bronze_incoming_data')
    .withColumn('hourly_timestamp', F.date_trunc('hour', F.from_unixtime('timestamp')))
    .groupBy('hourly_timestamp', 'turbine_id')
    .agg(*aggregations)
)

In [32]:
df.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save('../data/silver/silver_sensor_hourly')

**Note:** a more scalable solution would be to switch to streaming API and compute the aggregation with a ~3hours watermark and MERGE (upserting) the final output. For this demo clarity we we'll go with a full table update instead.

In [ ]:
# streaming_df = (
#     spark.readStream.format('delta')
#     .load('../data/bronze/bronze_incoming_data')
#     .withColumn('event_time', F.from_unixtime('timestamp'))
#     .withWatermark('event_time', '3 hours')  # watermark to handle late data
#     .withColumn('hourly_timestamp', F.date_trunc('hour', F.col('event_time')))
#     .groupBy('hourly_timestamp')
#     .agg(*aggregations)
# )


# # ---
# from delta.tables import DeltaTable

# def upsert_to_silver(microbatch_df, batch_id):
#     silver_path = '../data/silver/silver_sensor_hourly'
    
#     if DeltaTable.isDeltaTable(spark, silver_path):
#         silver_table = DeltaTable.forPath(spark, silver_path)
#         (
#             silver_table.alias("target")
#             .merge(
#                 microbatch_df.alias("source"),
#                 "target.hourly_timestamp = source.hourly_timestamp"
#             )
#             .whenMatchedUpdateAll()
#             .whenNotMatchedInsertAll()
#             .execute()
#         )
#     else:
#         # First time: just write the full table
#         microbatch_df.write.format("delta").save(silver_path)


# # ---
# query = (
#     streaming_df.writeStream
#     .outputMode("update")  # or "complete", depending on your aggregation
#     .foreachBatch(upsert_to_silver)
#     .option("checkpointLocation", "../checkpoints/silver_sensor_hourly")
#     .start()
# )

# query.awaitTermination()